In [6]:
import numpy as np

def calc_feat_mean_stds(name, x):
    means = np.mean(x, axis=0)
    stds = np.std(x, axis=0)
    arr = []
    for i, (mean,std) in enumerate(zip(means, stds)):
        arr.append({"feat_id": i, "mean":mean, "std":std})
    return arr

def calc_data_dist(name, x):
    arr = []
    
    values, bin_edges = np.histogram(x)
    sum_values = sum(values)
    for bin, value in zip(bin_edges[:-1], values):
        arr.append({"bin": bin, "value": value/sum_values, "name": name})
        
    return arr

def calc_all_data_dist_2d(name, x):
    arr = []
    step = int(x.shape[1]/8)  # we want to reduce to 8x8 grid
    
    for i in range(x.shape[1]//step):
        for j in range(x.shape[2]//step):
            values, bin_edges = np.histogram(x[:, i*step, j*step])

            for bin, value in zip(bin_edges[:-1], values):
                arr.append({"x": i, "y": j, "bin": bin, "value": value, "name": name})
        
    return arr   
  
def log_all_data_dist_2d(key, x):
    arr = calc_all_data_dist_2d(key, x)
    wandb.summary[key] = arr

def log_data_dist(key, x):
    arr = calc_data_dist(key, x)
    wandb.summary[key] = arr
    #wandb.summary[name] = wandb.Histogram(x)
    
def log_data_dists(key, x1, x2):
    arr1 = calc_data_dist("x1", x1)
    arr2 = calc_data_dist("x2", x1)
    
    wandb.summary[key] = np.concatenate((arr1, arr2))
    #wandb.summary[name] = wandb.Histogram(x)
    
def log_feat_mean_stds(key, x):
    arr = calc_feat_mean_stds(key, x)
    wandb.summary[key] = arr


def log_feats_mean_stds(key, x1, x2):
    arr1 = calc_feat_mean_stds("x1", x1)
    arr2 = calc_feat_mean_stds("x2", x1)
    
    wandb.summary[key] = np.concatenate((arr1, arr2))

def log_2d_data_dist(name, X_train):
    hists = []
    arr = []

    for i in range(28):
        for j in range(28):
            hist, bin_edges = np.histogram(X_train[:,i,j])
            for bin, value in zip(bin_edges[:-1], hist):
                arr.append({"x": i, "y": j, "bin": bin, "value": value})
            hists.append(arr)

    wandb.summary[name] = arr
    
def log_mnist_data(X_train, y_train, X_test, y_test):
    log_data_dist("x_train", X_train)
    log_data_dist("x_test", X_test)
    log_data_dist("y_train", y_train)
    log_data_dist("y_test", y_test)
    
    log_all_data_dist_2d("x_train_all", X_train)
    log_all_data_dist_2d("x_test_all", X_test)
    
    X_train_flat = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
    X_test_flat = X_test.reshape((X_test.shape[0], X_test.shape[1]*X_test.shape[2]))

    log_feat_mean_stds("x_train_mean_std", X_train_flat)
    log_feat_mean_stds("x_test_mean_std", X_test_flat)
    
    

In [5]:
# Original MNIST

from keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
import wandb
from wandb.keras import WandbCallback

# logging code
run = wandb.init(project="all-mnist", entity="l2k2")
config = run.config
wandb.summary["dataset"] = "orig-mnist"

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
labels = range(10)

num_classes = y_train_cat.shape[1]
log_mnist_data(X_train, y_train, X_test, y_test)

# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat), callbacks=[WandbCallback()])

KeyboardInterrupt: 

In [ ]:
# Fashion MNIST

from keras.utils import to_categorical
from keras.datasets import mnist, fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
import wandb
from wandb.keras import WandbCallback

# logging code
run = wandb.init(project="all-mnist", entity="l2k2")
config = run.config
wandb.summary["dataset"] = "fashion-mnist"

# load data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
labels = range(10)
log_mnist_data(X_train, y_train, X_test, y_test)

num_classes = y_train_cat.shape[1]

# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat), callbacks=[WandbCallback()])

In [7]:
# KMnist

from keras.utils import to_categorical
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
import wandb
from wandb.keras import WandbCallback
from utils import load_train_data, load_test_data

# logging code
run = wandb.init(project="all-mnist", entity="l2k2")
config = run.config
wandb.summary["dataset"] = "kmnist"

# load data
X_train, y_train = load_train_data("kmnist")
X_test, y_test = load_test_data("kmnist")
img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
labels = range(10)

log_mnist_data(X_train, y_train, X_test, y_test)

num_classes = y_train_cat.shape[1]

# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat), callbacks=[WandbCallback()])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 35us/step - loss: 9.3340 - acc: 0.4165 - val_loss: 9.3018 - val_acc: 0.4191
Epoch 2/10
60000/60000 [==============================] - 2s 30us/step - loss: 7.0869 - acc: 0.5568 - val_loss: 8.5077 - val_acc: 0.4689
Epoch 3/10
60000/60000 [==============================] - 2s 37us/step - loss: 6.6028 - acc: 0.5878 - val_loss: 8.4001 - val_acc: 0.4759
Epoch 4/10
60000/60000 [==============================] - 2s 32us/step - loss: 6.5283 - acc: 0.5926 - val_loss: 8.4922 - val_acc: 0.4701
Epoch 5/10
60000/60000 [==============================] - 2s 35us/step - loss: 6.3538 - acc: 0.6038 - val_loss: 8.2889 - val_acc: 0.4837
Epoch 6/10
60000/60000 [==============================] - 2s 36us/step - loss: 6.2625 - acc: 0.6098 - val_loss: 8.3129 - val_acc: 0.4822
Epoch 7/

In [123]:
# ASL (Sign Language) Mnist

from keras.utils import to_categorical
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
import wandb
from wandb.keras import WandbCallback
from utils import load_train_data, load_test_data
import signdata


# logging code
run = wandb.init(project="all-mnist", entity="l2k2")
config = run.config
wandb.summary["dataset"] = "asl-mnist"

# load data
(X_test, y_test) = signdata.load_test_data()
(X_train, y_train) = signdata.load_train_data()
img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
labels = range(10)

log_mnist_data(X_train, y_train, X_test, y_test)

num_classes = y_train_cat.shape[1]

# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat), callbacks=[WandbCallback()])

Train on 27455 samples, validate on 7172 samples
Epoch 1/10
13024/27455 [=============>................] - ETA: 4s - loss: 15.4118 - acc: 0.0437

KeyboardInterrupt: 

In [122]:
# Fashion MNIST - Broken!

from keras.utils import to_categorical
from keras.datasets import mnist, fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
import wandb
from wandb.keras import WandbCallback

# logging code
run = wandb.init(project="all-mnist", entity="l2k2")
config = run.config
wandb.summary["dataset"] = "fashion-mnist"


(X_train, y_train), (X_test, y_test) = mnist.load_data()

# load data
(X_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
labels = range(10)
log_mnist_data(X_train, y_train, X_test, y_test)

num_classes = y_train_cat.shape[1]

# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat), callbacks=[WandbCallback()])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 45us/step - loss: 12.9788 - acc: 0.1945 - val_loss: 14.4543 - val_acc: 0.0995
Epoch 2/10
60000/60000 [==============================] - 2s 35us/step - loss: 12.9105 - acc: 0.1990 - val_loss: 14.5247 - val_acc: 0.0978
Epoch 3/10
60000/60000 [==============================] - 2s 41us/step - loss: 12.9040 - acc: 0.1994 - val_loss: 14.4830 - val_acc: 0.1000
Epoch 4/10
60000/60000 [==============================] - 2s 39us/step - loss: 12.9179 - acc: 0.1985 - val_loss: 14.4881 - val_acc: 0.1003
Epoch 5/10
60000/60000 [==============================] - 3s 51us/step - loss: 12.9131 - acc: 0.1988 - val_loss: 14.4982 - val_acc: 0.1005
Epoch 6/10
60000/60000 [==============================] - 3s 53us/step - loss: 12.9028 - acc: 0.1995 - val_loss: 14.4743 - val_acc: 0.1012
Epoch 7/10
60000/60000 [==============================] - 2s 39us/step - loss: 12.8988 - acc: 0.1997 - val_loss: 14.